### Importing Libraries

In [1]:
import os
import re
import getpass
import pandas as pd
from io import StringIO
from langchain_google_genai.chat_models import ChatGoogleGenerativeAI
from langchain.agents import initialize_agent, Tool, AgentType
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

e:\TechM\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
e:\TechM\.venv\Lib\site-packages\langchain\chains\api\base.py:56: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_community.utilities.requests import TextRequestsWrapper


### Configuring the gemini api

In [2]:
os.environ["GOOGLE_API_KEY"] = "Your gemini key here"

In [3]:
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass("Provide your Google API key")

In [4]:
llm=ChatGoogleGenerativeAI(model='gemini-1.5-flash')

### Logic for evaluation tool

This function extracts questions and marks assigned to student from the dataset

In [5]:
def reader(row):
    query = (
        f"Your task is to extract the questions along with the marks assigned to that question and VERY IMPORTANT (full answers) of students present in the string. {row}",
        "You have to extract full answers. VERY IMPORTANT: DON'T PRINT ANYTHING EXCEPT EXTRACTED CONTENT."
    )
    result = llm.invoke(query) 
    return result.content  

This function evaluates the answers and gives marks to each student fo reach question

In [6]:
def evaluator(input_str):
    query = (
        f"Your task is to evaluate answers and give marks as output along with question number. {input_str}. Give marks accordingly, not just 10 or 0 and make sure to give fair marks for all students (VERY IMPORTANT).",
        "Just give output as marks of each question. In a csv format where the question is first column and score is second",
       
    )
    result = llm.invoke(query)  
    return result.content 

This function provides insights to each student on each question why the marks got deducted and if the student scored full marks it just returns well done

In [7]:
def insights(input_str):
    query=(
        f"Your task is to provide insights on the answers and if the answer is correct just return well done {input_str}",
        "Just give output as insights of each question and nothing else in insight column. In a csv format where the question number is first column and insights is second(should contain only one line)",
        "VERY IMPORTANT (the provided insight must be short)"
    )
    result=llm.invoke(query)
    return result.content

This fucntion is used to format the text generated by the llm

In [8]:
def remove_symbols(text):
    cleaned_text = re.sub(r'[^\w\s]', '/', text)
    return cleaned_text

This function pass the data to the above functions row wise from the uploaded excel file and then convert the data into a dataframe which has 2 columns one is question number and other is score

In [9]:
df_final_marks=None
def iterator1(path):
    global df_final_marks
    all_records = []
    df = pd.read_csv(path)
    for index, row in df.iterrows():
        extracted_content = reader(row)  
        evaluated_marks = evaluator(extracted_content)  
        cleaned_text = remove_symbols(evaluated_marks)  
        lines = cleaned_text.strip().split('\n')[1:]  
        records = [line.split('/') for line in lines]
        all_records.extend(records)
    df_final_marks = pd.DataFrame(all_records, columns=['Question', 'Score'])
    df_final_marks['Score'] = pd.to_numeric(df_final_marks['Score'])
    df_final_marks=df_final_marks
    print(df_final_marks)


This function pass the data to the above functions row wise from the uploaded excel file and then convert the data into a dataframe which has 2 columns one is question number and other is insisghts

In [10]:
def iterator2(path):
    df=pd.read_csv(path)
    all_records1=[]
    global df_final_insights
    for index, row in df.iterrows():
        extracted_content = reader(row)  
        evaluated_marks = insights(extracted_content)  
        cleaned_text = remove_symbols(evaluated_marks) 
        lines=cleaned_text.strip().split('\n')[1:]
        records=[line.split('/')[:2] for line in lines]
        all_records1.extend(records)
    df_final_insights=pd.DataFrame(all_records1,columns=['Question','Insights'])
    print(df_final_insights)

This function passes the path of the file to both evaluation and insight function

In [11]:
def readcsv(path):
    iterator1(path)
    iterator2(path)
    return path

This function is to merge and format and store it to a pandas dataframe

In [12]:
def create_database(path):
    global final_df_data
    global df_final_insights
    df_final_insights=df_final_insights.drop('Question',axis=1)
    df_data=pd.concat([df_final_marks,df_final_insights],axis=1)
    df=pd.read_csv(path)
    username_column = df['Username']
    score_columns = df.filter(like='[Score]')
    df_extracted = pd.concat([username_column, score_columns], axis=1)
    df_extracted.loc[:, df_extracted.columns != 'Username'] = df_extracted.loc[:, df_extracted.columns != 'username'].applymap(lambda x: ''.join(filter(str.isdigit, str(x))))
    total_rows = len(df_extracted)
    df=df_extracted.merge(df_data,how='cross')
    m = total_rows
    first_user_email = df['Username'].iloc[0]
    first_user_data = df[df['Username'] == first_user_email].head(m)
    other_users_data = df[df['Username'] != first_user_email].groupby('Username').tail(m)
    final_df_data = pd.concat([first_user_data, other_users_data])
    final_df_data.reset_index(drop=True, inplace=True)
    print(final_df_data)

### Defining tools for langchain tool

In [13]:
evaluator_tool = Tool(
    name='Response Evaluation Tool',
    func=readcsv,
    description='The task is to pass the path to the readcsv function'
)
database_tool = Tool(
    name='Database Tool',
    func=create_database,  
    description='The task is to pass the path to the create_database function'
)

In [14]:
tools = [evaluator_tool,database_tool]

In [15]:
memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=3,
    return_messages=True
)

C:\Users\ishaa\AppData\Local\Temp\ipykernel_30316\3940763615.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(


In [16]:
conversational_agent_evaluator=initialize_agent(
    agent='chat-conversational-react-description',
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
    memory=memory
)

C:\Users\ishaa\AppData\Local\Temp\ipykernel_30316\2219053028.py:1: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use :meth:`~Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc.` instead.
  conversational_agent_evaluator=initialize_agent(


### Logic for custom agent run

In [17]:
def custom_agent_run(query):
    result1 = evaluator_tool.run(query)  
    result2 = database_tool.run(result1)  
    return result2  
path=input()
output = custom_agent_run(path)
print(f"Final output: {output}")


     Question  Score
0  Question 1     10
1  Question 2      0
2  Question 1     10
3  Question 2     10
  Question                                  Insights
0        1                                Well done 
1        2  The answer does not address the question
2        1                                 Well done
3        2                                 Well done
                      Username What is the SI unit of force? [Score]  \
0  malhotraishaan857@gmail.com                                    10   
1  malhotraishaan857@gmail.com                                    10   
2         malhotra90@gmail.com                                    10   
3         malhotra90@gmail.com                                    10   

  Explain the difference between mass and weight? [Score]    Question  Score  \
0                                                 10       Question 1     10   
1                                                 10       Question 2      0   
2                            

C:\Users\ishaa\AppData\Local\Temp\ipykernel_30316\2887323470.py:10: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_extracted.loc[:, df_extracted.columns != 'Username'] = df_extracted.loc[:, df_extracted.columns != 'username'].applymap(lambda x: ''.join(filter(str.isdigit, str(x))))


### For students to view there marks 

In [19]:
input_username = 'malhotraishaan857@gmail.com'
filtered_df = final_df_data[final_df_data['Username'] == input_username]
total_marks=filtered_df['Score'].sum()
print(f"The total marks scored are {total_marks}")
filtered_df

The total marks scored are 10


,Username,What is the SI unit of force? [Score],Explain the difference between mass and weight? [Score],Question,Score,Insights
0,malhotraishaan857@gmail.com,10,10,Question 1,10,Well done
1,malhotraishaan857@gmail.com,10,10,Question 2,0,The answer does not address the question
